In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

tqdm.pandas()

In [3]:
structure = pd.read_csv("kaggle/structure.csv")
targets = pd.read_csv("kaggle/targets.csv")
df = pd.read_csv("kaggle/user_activity.csv")

In [4]:
costed_steps = structure[structure.step_cost > 0]

In [9]:
df[(df.step_id.isin(costed_steps.step_id)) & (df.user_id == 16953)]

,user_id,action,step_id,time,step_type,step_cost
154347,16953,discovered,4289,1410894805,choice,1
154348,16953,viewed,4289,1410894805,choice,1
154355,16953,started_attempt,4289,1410894809,choice,1
361732,16953,discovered,2918,1411490384,code,2
361733,16953,viewed,2918,1411490384,code,2
361737,16953,discovered,2919,1411490389,code,2
361738,16953,viewed,2919,1411490389,code,2
361740,16953,discovered,4969,1411490391,code,2
361741,16953,viewed,4969,1411490391,code,2
361760,16953,discovered,7422,1411490400,code,2


In [66]:
def f(x):
    passed = x[x.action == "passed"]
    not_passed = x[~(x.step_id.isin(passed.step_id))]
    
    #lol-comprehension
    avg_attempts = np.mean([t.time.values.size for t in [x[(x.action == "started_attempt") & (x.step_id == id_)] for id_ in x.step_id.unique()]])
    
    not_passed = not_passed.drop_duplicates("step_id")
#     print(not_passed)

    days = 10*3600*24
    border = passed.time.max() - days
    passed_steps = len([1 for t in passed.time if t >= border])
    skipped_steps = not_passed.time.values.size
    
    passed_balls = passed.step_cost.sum()
    viewed_balls = not_passed[not_passed.action == "viewed"].step_cost.sum()
    if not viewed_balls:
        q = 0
    else:
        q = passed_balls / viewed_balls
    
    res = [passed_steps, skipped_steps, avg_attempts, q]
    return res

# TODO: Number of skipped steps and substraction of passed and skipped

In [67]:
x = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").get_group(16953)

In [68]:
f(x)

[10, 4, 1.8571428571428572, 7.0]

In [69]:
X_t = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").progress_apply(f)
Y_t = targets[targets.user_id.isin(X_t.index)]





  0%|          | 0/9426 [00:00<?, ?it/s]



  0%|          | 2/9426 [00:00<12:19, 12.74it/s]



  0%|          | 3/9426 [00:00<13:31, 11.61it/s]



  0%|          | 8/9426 [00:00<10:30, 14.94it/s]



  0%|          | 13/9426 [00:00<08:17, 18.91it/s]



  0%|          | 19/9426 [00:00<06:42, 23.40it/s]



  0%|          | 23/9426 [00:00<06:15, 25.07it/s]



  0%|          | 31/9426 [00:00<04:57, 31.56it/s]



  0%|          | 37/9426 [00:00<04:35, 34.05it/s]



  0%|          | 45/9426 [00:01<03:50, 40.69it/s]



  1%|          | 52/9426 [00:01<03:25, 45.64it/s]



  1%|          | 58/9426 [00:01<03:20, 46.82it/s]



  1%|          | 64/9426 [00:01<04:16, 36.49it/s]



  1%|          | 69/9426 [00:01<04:01, 38.73it/s]



  1%|          | 76/9426 [00:01<03:32, 43.96it/s]



  1%|          | 82/9426 [00:01<03:28, 44.84it/s]



  1%|          | 87/9426 [00:02<03:40, 42.32it/s]



  1%|          | 94/9426 [00:02<03:15, 47.62it/s]



  1%|          | 100/9426 [00:02<03:28, 44.70it/s]





In [70]:
X_t.values, Y_t.passed.values

(array([[0, 68, 0.0, 0.0], [9, 0, 2.3333333333333335, 0],
        [9, 0, 2.6666666666666665, 0], ..., [2, 1, 3.3333333333333335, 2.0],
        [0, 2, 1.0, 0.0], [3, 0, 6.0, 0]], dtype=object),
 array([0, 0, 0, ..., 0, 0, 0]))

In [71]:
X = np.array([np.array(x) for x in X_t.values]).reshape(-1, 4)
Y = Y_t.passed.ravel()

In [72]:
X

array([[  0.        ,  68.        ,   0.        ,   0.        ],
       [  9.        ,   0.        ,   2.33333333,   0.        ],
       [  9.        ,   0.        ,   2.66666667,   0.        ],
       ..., 
       [  2.        ,   1.        ,   3.33333333,   2.        ],
       [  0.        ,   2.        ,   1.        ,   0.        ],
       [  3.        ,   0.        ,   6.        ,   0.        ]])

In [88]:
classifier = SGDClassifier(loss="squared_hinge")
classifier.coef_ = [5.22854234, -7.15484741, -6.31902466, -3.07028643]
classifier.intercept_ = [-36.91845681]

In [89]:
best_params = [None, None]
scores = []
best_score = 0
for _ in tqdm_notebook(range(1000)):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
    classifier.fit(X_train, Y_train)
    Y_pred = classifier.predict(X_test)
    if Y_pred.sum() > 0:
        score = f1_score(Y_test, Y_pred)
        if score > best_score:
            print(Y_pred.sum())
            best_score = max(best_score, score)
            best_params = [classifier.coef_, classifier.intercept_] 
        scores.append(score)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

116


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

...

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [90]:
np.median(scores)
np.mean(scores)

0.2512095360523548

0.21846625256572105

In [80]:
best_params

[array([[ 2.3696734 , -3.32163541, -0.57359424, -7.78178374]]),
 array([-24.63634201])]

In [81]:
def prepare(df, num_features):
    X = df.groupby("user_id").progress_apply(f)
    return X.index, np.array([np.array(x) for x in X.values]).reshape(-1, num_features)

In [82]:
user_ids, X_val = prepare(pd.read_csv("kaggle/user_activity_test.csv"), 4)





  0%|          | 0/4256 [00:00<?, ?it/s]



  0%|          | 4/4256 [00:00<01:53, 37.34it/s]



  0%|          | 7/4256 [00:00<04:28, 15.84it/s]



  0%|          | 9/4256 [00:00<05:05, 13.92it/s]



  0%|          | 11/4256 [00:00<06:04, 11.65it/s]



  0%|          | 13/4256 [00:01<05:19, 13.30it/s]



  0%|          | 15/4256 [00:01<11:24,  6.20it/s]



  0%|          | 21/4256 [00:02<08:42,  8.11it/s]



  1%|          | 23/4256 [00:02<08:35,  8.20it/s]



  1%|          | 25/4256 [00:02<08:33,  8.25it/s]



  1%|          | 27/4256 [00:02<08:44,  8.06it/s]



  1%|          | 30/4256 [00:02<07:19,  9.61it/s]



  1%|          | 32/4256 [00:03<08:33,  8.23it/s]



  1%|          | 34/4256 [00:03<08:00,  8.78it/s]



  1%|          | 36/4256 [00:03<08:09,  8.63it/s]



  1%|          | 38/4256 [00:03<08:56,  7.86it/s]



  1%|          | 41/4256 [00:04<07:45,  9.05it/s]



  1%|          | 43/4256 [00:04<06:59, 10.04it/s]



  1%|          | 46/4256 [00:04<05:50, 12.00it/s]



 

In [83]:
X_train[5], X_val[5]

(array([ 2. ,  0. ,  4.5,  0. ]),
 array([ 30.        ,   0.        ,   0.16666667,   0.        ]))

In [84]:
predicted = classifier.predict(X_val)

In [85]:
res = pd.DataFrame({"user_id": user_ids, "passed": predicted}).set_index("user_id")

In [86]:
res.passed.value_counts()

1    2715
0    1540
Name: passed, dtype: int64

In [87]:
res.to_csv("result.csv")